## 1. Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import boto3
from botocore.config import Config
import dask.dataframe as dd

## 1. Load and Prepare Data

In [5]:
import os
import boto3
from botocore.config import Config

def download_from_r2(object_name, local_path, bucket_name="bookdbio"):
    # ensure parent dir exists
    parent_dir = os.path.dirname(local_path)
    if parent_dir and not os.path.isdir(parent_dir):
        os.makedirs(parent_dir, exist_ok=True)

    s3 = boto3.client('s3',
        endpoint_url = f"https://a9a190ee80813000e18bacf626b1281b.r2.cloudflarestorage.com/",
        aws_access_key_id = '85fec6dd1268801ac8c1c59175ba0b76',
        aws_secret_access_key = '798b753bab748f2c7f5e0f46fd6506b7f0b206e362b1e00055d060a72b88d55d',
        config = Config(signature_version='s3v4')
   )

    try:
        s3.download_file(bucket_name, object_name, local_path)
        print(f"Successfully downloaded {object_name} to {local_path}")
    except Exception as e:
        print(f"Download failed for {object_name}: {e}")

In [2]:
def list_bucket_contents(bucket_name="bookdbio"):
    """List all objects in the R2 bucket"""
    s3 = boto3.client('s3',
        endpoint_url = f"https://a9a190ee80813000e18bacf626b1281b.r2.cloudflarestorage.com/",
        aws_access_key_id = '85fec6dd1268801ac8c1c59175ba0b76',
        aws_secret_access_key = '798b753bab748f2c7f5e0f46fd6506b7f0b206e362b1e00055d060a72b88d55d',
        config = Config(signature_version='s3v4')
   )
    
    try:
        response = s3.list_objects_v2(Bucket=bucket_name)
        if 'Contents' in response:
            print("Available files in bucket:")
            for obj in response['Contents']:
                print(f"- {obj['Key']}")
        else:
            print("Bucket is empty")
    except Exception as e:
        print(f"Error listing bucket contents: {e}")

In [3]:
list_bucket_contents()

Available files in bucket:
- data/author_id_map.csv
- data/authors.parquet
- data/book_id_map.csv
- data/book_texts.parquet
- data/book_texts_reduced.parquet
- data/books.parquet
- data/books_dedup.parquet
- data/books_triplets.parquet
- data/books_triplets_reduced.parquet
- data/books_works.parquet
- data/interactions.parquet
- data/interactions_dedup.parquet
- data/interactions_prepared_ncf.parquet
- data/interactions_prepared_ncf_reduced.parquet
- data/item_id_map_reduced.csv
- data/new_authors.parquet
- data/new_books.parquet
- data/reduced_book_ids.csv
- data/reduced_books.parquet
- data/reduced_interactions.parquet
- data/reduced_reviews.parquet
- data/reduced_user_ids.csv
- data/reviews_dedup.parquet
- data/user_id_map.csv
- data/user_id_map_reduced.csv
- db/bookdb.sql
- embeddings/SBERT_embeddings.parquet
- embeddings/gmf_book_embeddings.parquet
- embeddings/gmf_user_embeddings.parquet


In [6]:
download_from_r2("data/reduced_books.parquet", "data/reduced_books.parquet")

Successfully downloaded data/reduced_books.parquet to data/reduced_books.parquet


In [7]:
books_df = dd.read_parquet("data/reduced_books.parquet")

In [8]:
books_df.head() 

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",,false,3.49,B002ENBLOK,...,7,,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever
15,0800759494,2885,[],US,,"[{'count': '9381', 'name': 'to-read'}, {'count...",,false,3.91,B00B853QPM,...,,,,https://www.goodreads.com/book/show/89375.90_M...,https://s.gr-assets.com/assets/nophoto/book/11...,89375,68157,2957021,90 Minutes in Heaven: A True Story of Death an...,90 Minutes in Heaven: A True Story of Death an...
479,,346,[274178],US,en-GB,"[{'count': '6001', 'name': 'to-read'}, {'count...",B0055Q8HDG,true,3.86,B0055Q8HDG,...,6,,2011,https://www.goodreads.com/book/show/11731782-c...,https://images.gr-assets.com/books/1352764436m...,11731782,5125,16680541,"Collide (Collide, #1)","Collide (Collide, #1)"
583,0395083621,1396,[],US,en-US,"[{'count': '549', 'name': 'history'}, {'count'...",,false,3.13,B008QI6EHQ,...,,,,https://www.goodreads.com/book/show/54270.Mein...,https://images.gr-assets.com/books/1395618385m...,54270,16837,2049624,Mein Kampf,Mein Kampf
807,0060773758,1223,[163036],US,en-US,"[{'count': '7188', 'name': 'to-read'}, {'count...",,false,3.91,B000GCFWXW,...,10,,2005,https://www.goodreads.com/book/show/38568.A_Qu...,https://images.gr-assets.com/books/1410129015m...,38568,32140,2621331,A Quick Bite (Argeneau #1),A Quick Bite (Argeneau #1)


In [9]:
download_from_r2("data/reduced_interactions.parquet", "data/reduced_interactions.parquet")

Successfully downloaded data/reduced_interactions.parquet to data/reduced_interactions.parquet


In [10]:
interactions_df = dd.read_parquet("data/reduced_interactions.parquet")

In [11]:
interactions_df.head()

,user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
0,8842281e1d1347389f2ab93d60773d4d,6480781,c8676124d8829874576fcb868af89315,True,5,,Mon Mar 20 23:58:16 -0700 2017,Wed Mar 22 11:47:49 -0700 2017,,
1,8842281e1d1347389f2ab93d60773d4d,29584452,dacadc8f32e80bbdb5cd052a84ad5c63,True,4,,Tue Nov 15 09:23:32 -0800 2016,Tue Dec 13 11:00:02 -0800 2016,Wed Nov 23 00:00:00 -0800 2016,Fri Nov 18 00:00:00 -0800 2016
2,8842281e1d1347389f2ab93d60773d4d,28119237,7a8dc8ab7f3c0084be8150d7e5bd40cb,True,4,A fascinating book about community and belongi...,Thu Sep 22 16:19:12 -0700 2016,Thu Sep 22 16:19:12 -0700 2016,Tue Nov 22 00:00:00 -0800 2016,
3,8842281e1d1347389f2ab93d60773d4d,186074,1d12addadc0c737dcd29c362c936a266,True,5,,Tue Jul 12 19:21:10 -0700 2016,Wed Mar 22 11:47:46 -0700 2017,,
4,8842281e1d1347389f2ab93d60773d4d,15839976,c7baa53f6f7d554ed9c3859f0d400d19,True,5,,Tue May 10 11:34:44 -0700 2016,Wed Mar 22 11:47:45 -0700 2017,,


In [12]:
# Count the number of unique user IDs in the interactions dataframe
unique_users_count = interactions_df['user_id'].nunique().compute()
print(f"Number of unique user IDs: {unique_users_count}")


Number of unique user IDs: 205242


### Creating user context

In [13]:
def analyze_dataframe(df):
    # Get column info
    cols = df.columns
    dtypes = df.dtypes
    
    # Calculate total rows
    total_rows = len(df.compute())
    
    # Initialize lists to store results
    results = []
    
    # Analyze each column
    for col in cols:
        # Count non-null values
        non_null_count = df[col].count().compute()
        null_count = total_rows - non_null_count
        null_percentage = (null_count / total_rows) * 100
        
        results.append({
            'Column': col,
            'Data Type': str(dtypes[col]),
            'Non-Null Count': non_null_count,
            'Null Count': null_count,
            'Null Percentage': f'{null_percentage:.2f}%'
        })
    
    # Convert results to pandas DataFrame for better display
    results_df = pd.DataFrame(results)
    return results_df.sort_values('Null Percentage', ascending=False)

# Display the analysis
print("DataFrame Analysis:")
display(analyze_dataframe(books_df))

DataFrame Analysis:


,Column,Data Type,Non-Null Count,Null Count,Null Percentage
0,isbn,string,17663,0,0.00%
15,publisher,string,17663,0,0.00%
27,title,string,17663,0,0.00%
26,work_id,string,17663,0,0.00%
25,ratings_count,string,17663,0,0.00%
24,book_id,int64,17663,0,0.00%
23,image_url,string,17663,0,0.00%
22,url,string,17663,0,0.00%
21,publication_year,string,17663,0,0.00%
20,edition_information,string,17663,0,0.00%


In [14]:
books_df.set_index('book_id', inplace=True)


,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,ratings_count,work_id,title,title_without_series
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,string,string,object,string,string,object,string,string,string,string,object,string,string,string,object,string,string,string,string,string,string,string,string,string,string,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [15]:
books_df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",,false,3.49,B002ENBLOK,...,7,,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever
15,0800759494,2885,[],US,,"[{'count': '9381', 'name': 'to-read'}, {'count...",,false,3.91,B00B853QPM,...,,,,https://www.goodreads.com/book/show/89375.90_M...,https://s.gr-assets.com/assets/nophoto/book/11...,89375,68157,2957021,90 Minutes in Heaven: A True Story of Death an...,90 Minutes in Heaven: A True Story of Death an...
479,,346,[274178],US,en-GB,"[{'count': '6001', 'name': 'to-read'}, {'count...",B0055Q8HDG,true,3.86,B0055Q8HDG,...,6,,2011,https://www.goodreads.com/book/show/11731782-c...,https://images.gr-assets.com/books/1352764436m...,11731782,5125,16680541,"Collide (Collide, #1)","Collide (Collide, #1)"
583,0395083621,1396,[],US,en-US,"[{'count': '549', 'name': 'history'}, {'count'...",,false,3.13,B008QI6EHQ,...,,,,https://www.goodreads.com/book/show/54270.Mein...,https://images.gr-assets.com/books/1395618385m...,54270,16837,2049624,Mein Kampf,Mein Kampf
807,0060773758,1223,[163036],US,en-US,"[{'count': '7188', 'name': 'to-read'}, {'count...",,false,3.91,B000GCFWXW,...,10,,2005,https://www.goodreads.com/book/show/38568.A_Qu...,https://images.gr-assets.com/books/1410129015m...,38568,32140,2621331,A Quick Bite (Argeneau #1),A Quick Bite (Argeneau #1)


In [16]:

def extract_genres(popular_shelves):
    """
    Extracts potential genres from a list of popular shelves dictionaries,
    adding only the base genre keyword found.

    Args:
        popular_shelves: A list of dictionaries, where each dictionary has
                         'count' and 'name' keys.

    Returns:
        A list of unique base genre names found, or an empty list on error.
    """
    try:
        if not isinstance(popular_shelves, np.ndarray) or len(popular_shelves) == 0:
            return []
        
        # Use a set to store unique base genres found
        found_genres = set() 
        
        genre_keywords = [
            'action', 'adventure', 'comedy', 'crime', 'mystery', 'textbook', 'children', 'mathematics', 'fantasy',
            'historical', 'horror', 'romance', 'satire', 'science fiction',
            'scifi', 'speculative fiction', 'thriller', 'western', 'paranormal',
            'dystopian', 'urban fantasy', 'contemporary', 'young adult', 'ya',
            'middle grade', 'children\'s', 'literary fiction', 'magic realism',
            'historical fiction', 'gothic', 'suspense', 'biography', 'memoir',
            'nonfiction', 'poetry', 'drama', 'historical romance',
            'fantasy romance', 'romantic suspense', 'science fiction romance',
            'contemporary romance', 'paranormal romance', 'epic fantasy',
            'dark fantasy', 'sword and sorcery', 'steampunk', 'cyberpunk',
            'apocalyptic', 'post-apocalyptic', 'alternate history',
            'superhero', 'mythology', 'fairy tales', 'folklore', 'war',
            'military fiction', 'spy fiction', 'political fiction', 'social science fiction',
            'techno-thriller', 'medical thriller', 'legal thriller',
            'psychological thriller', 'cozy mystery', 'hardboiled', 'noir',
            'coming-of-age', 'lgbtq+', 'christian fiction', 'religious fiction',
            'humor', 'travel', 'food', 'cooking', 'health', 'self-help',
            'business', 'finance', 'history', 'science', 'technology', 'nature',
            'art', 'music', 'philosophy', 'education', 'true crime', 'spiritual',
            'anthology', 'short stories', 'plays', 'screenplays', 'graphic novel',
            'comics', 'manga', 'erotica', 'new adult', 'chick lit', 'womens fiction',
            'sports fiction', 'family saga', ' Regency romance', 'literature'
        ]
        # Sort keywords by length descending to match longer phrases first (e.g., "science fiction" before "science")
        genre_keywords.sort(key=len, reverse=True)

        ignore_keywords = ['to-read', 'owned', 'hardcover', 'shelfari-favorites', 'series', 'might-read',
                           'dnf-d', 'hambly-barbara', 'strong-females', 'first-in-series',
                           'no-thanks-series-collections-boxes', 'entertaining-but-limited',
                           'kate-own', 'e-book', 'compliation', 'my-books',
                           'books-i-own-but-have-not-read', 'everything-owned', 'books-to-find',
                           'i-own-it', 'favorite', 'not-read', 'read-some-day', 'library',
                           'audiobooks', 'status-borrowed', 'owned-books',
                           'spec-fic-awd-locus-nom', '01', 'hardbacks', 'paper', 'german',
                           'hardback', 'physical-scifi-fantasy', 'childhood-favorites',
                           'bundle-same-author', 'aa-sifi-fantasy', 'ready-to-read',
                           'bought-on-flee-markets', 'fantasy-general', 'hardcopy', 'box-2',
                           'unfinished', 'magic', 'duplicates', 'favorites', 'books-i-own',
                           'fantasy-classic', 'own-hard-copy', 'fantasy-read',
                           'book-club-edition', 'sci-fi-or-fantasy', 'fiction-fantasy',
                           'fiction-literature-poetry', 'paused-hiatus', 'status—borrowed',
                           'recs-fantasy', 'fantasy-scifi', 'omnibus', 'speculative',
                           'sf--fantasy', 'in-my-home-library', 'fant-myth-para-vamps',
                           'read-in-my-20s']

        for shelf in popular_shelves:
            if not isinstance(shelf, dict) or 'name' not in shelf:
                continue
            
            shelf_name = shelf['name'].lower().strip() # Normalize shelf name

            # Skip if shelf name contains any ignore keywords
            if any(ignore in shelf_name for ignore in ignore_keywords):
                continue

            # Check if any genre keyword is present in the shelf name
            for keyword in genre_keywords:
                # Use word boundaries or careful checks to avoid partial matches (e.g., 'art' in 'heart')
                # Simple substring check for now, might need refinement depending on data
                if keyword in shelf_name: 
                    found_genres.add(keyword) # Add the base keyword
                    # Optional: break here if you only want the first/longest match per shelf
                    # break 

        return sorted(list(found_genres))
    except Exception as e:
        print(f"Error in extract_genres function: {e}")
        # Log the error message
        logging.error("Error in extract_genres function", exc_info=True)
        return []

In [17]:
books_df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
3,0743294297,3282,[],US,eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",,false,3.49,B002ENBLOK,...,7,,2009,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184,6243154,Best Friends Forever,Best Friends Forever
15,0800759494,2885,[],US,,"[{'count': '9381', 'name': 'to-read'}, {'count...",,false,3.91,B00B853QPM,...,,,,https://www.goodreads.com/book/show/89375.90_M...,https://s.gr-assets.com/assets/nophoto/book/11...,89375,68157,2957021,90 Minutes in Heaven: A True Story of Death an...,90 Minutes in Heaven: A True Story of Death an...
479,,346,[274178],US,en-GB,"[{'count': '6001', 'name': 'to-read'}, {'count...",B0055Q8HDG,true,3.86,B0055Q8HDG,...,6,,2011,https://www.goodreads.com/book/show/11731782-c...,https://images.gr-assets.com/books/1352764436m...,11731782,5125,16680541,"Collide (Collide, #1)","Collide (Collide, #1)"
583,0395083621,1396,[],US,en-US,"[{'count': '549', 'name': 'history'}, {'count'...",,false,3.13,B008QI6EHQ,...,,,,https://www.goodreads.com/book/show/54270.Mein...,https://images.gr-assets.com/books/1395618385m...,54270,16837,2049624,Mein Kampf,Mein Kampf
807,0060773758,1223,[163036],US,en-US,"[{'count': '7188', 'name': 'to-read'}, {'count...",,false,3.91,B000GCFWXW,...,10,,2005,https://www.goodreads.com/book/show/38568.A_Qu...,https://images.gr-assets.com/books/1410129015m...,38568,32140,2621331,A Quick Bite (Argeneau #1),A Quick Bite (Argeneau #1)


#### Creating shortened book df texts

We will create a shortened book dataframe contining only the important information about the book needed for creating user context.

In [18]:
download_from_r2("data/new_authors.parquet", "data/new_authors.parquet")

Successfully downloaded data/new_authors.parquet to data/new_authors.parquet


In [19]:
authors_df = dd.read_parquet("data/new_authors.parquet")

In [20]:
# Create reduced DataFrame
reduced_books_df = books_df[['book_id', 'title', 'description']].copy()

# Modify extract_genres to return a string instead of a list
def extract_genres_string(shelves):
    genres = extract_genres(shelves)
    return ','.join(genres) if genres else ''

# Apply the modified function to get string representation of genres
reduced_books_df['genre'] = books_df['popular_shelves'].apply(extract_genres_string)

# Convert authors to string representation as well
def get_author_names(author_ids):
    author_names = []
    for author_id in author_ids:
        try:
            name = authors_df.loc[authors_df['author_id'] == author_id]['name'].compute().values[0]
            author_names.append(name)
        except:
            continue
    return ','.join(author_names)

reduced_books_df['authors'] = books_df['authors'].apply(get_author_names)

# Display sample of the reduced DataFrame
print("\nSample of reduced books DataFrame:")
print(reduced_books_df.head())

# Display genre distribution (need to split the strings for counting)
print("\nGenre distribution:")
genre_counts = reduced_books_df['genre'].apply(lambda x: x.split(',') if x else []).explode().value_counts()
print(genre_counts)

/Users/anze/Documents/University/Y2S2/AI Machine Learning Foundations/Final Project/.venv/lib/python3.13/site-packages/dask/dataframe/dask_expr/_collection.py:4392: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('popular_shelves', 'object'))

  warnings.warn(meta_warning(meta))
/Users/anze/Documents/University/Y2S2/AI Machine Learning Foundations/Final Project/.venv/lib/python3.13/site-packages/dask/dataframe/dask_expr/_collection.py:4392: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this messa


Sample of reduced books DataFrame:
      book_id                                              title  \
3     6066819                               Best Friends Forever   
15      89375  90 Minutes in Heaven: A True Story of Death an...   
479  11731782                              Collide (Collide, #1)   
583     54270                                         Mein Kampf   
807     38568                         A Quick Bite (Argeneau #1)   

                                           description  \
3    Addie Downs and Valerie Adler were eight when ...   
15   As he is driving home from a minister's confer...   
479  Sherry has always known there was something ou...   
583  Madman, tyrant, animal - history has given Ado...   
807  That hot guy tied to Lissianna Argeneau's bed?...   

                                                 genre  \
3    coming-of-age,contemporary,drama,humor,mystery...   
15     biography,memoir,nonfiction,self-help,spiritual   
479  contemporary,dystopian,fant

/Users/anze/Documents/University/Y2S2/AI Machine Learning Foundations/Final Project/.venv/lib/python3.13/site-packages/dask/dataframe/dask_expr/_collection.py:4392: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('genre', 'object'))

  warnings.warn(meta_warning(meta))


In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import psutil

# Monitor memory usage
def print_memory_usage():
    process = psutil.Process()
    print(f"Memory usage: {process.memory_info().rss / 1024 / 1024:.2f} MB")

# Before processing
print_memory_usage()

# Set up progress bar
with ProgressBar():
    # Process your DataFrame
    result = reduced_books_df.compute()  # or whatever operation you're doing
    
    # Save to parquet with optimizations
    result.to_parquet(
        'data/reduce_books_df.parquet',
        compression='snappy',
        index=False,
        engine='pyarrow'
    )

# After processing
print_memory_usage()

In [ ]:
test = dd.read_parquet('data/reduce_books_df.parquet')

In [ ]:
# Create a DataFrame with user_id and their read books sorted by rating
def create_user_books_df(interactions_df):
    """
    Create a DataFrame with user_id and their read books sorted by rating.
    
    Args:
        interactions_df: DataFrame containing user-book interactions
        
    Returns:
        DataFrame with columns 'user_id' and 'books_read'
    """
    import pandas as pd
    
    # Get all unique users
    unique_users = interactions_df['user_id'].unique().compute().tolist()
    
    # Create a list to store user_id and read_books pairs
    user_books_data = []
    
    # For each user, get their read books sorted by rating
    for user_id in unique_users:
        # Filter interactions for the specific user
        user_interactions = interactions_df[interactions_df['user_id'] == user_id]
        
        # Sort by rating in descending order
        sorted_interactions = user_interactions.sort_values(by='rating', ascending=False)
        
        # Get all book_ids
        read_books = sorted_interactions['book_id'].compute().tolist()
        
        user_books_data.append({
            'user_id': user_id,
            'books_read': read_books
        })
    
    # Create DataFrame from the collected data
    user_books_df = pd.DataFrame(user_books_data)
    
    return user_books_df

# Create the user_books_df with all users
user_books_df = create_user_books_df(interactions_df)

# Display sample of the DataFrame
print("\nSample of user_books_df:")
print(user_books_df.head())


In [26]:
user_books_df.shape

(100, 2)

In [25]:
# Sample 50,000 users from the pandas DataFrame
sampled_users_book_pd = user_books_df.sample(n=50000, random_state=42)

# If you need it back as a Dask DataFrame
sampled_users_book = dd.from_pandas(sampled_users_book_pd, npartitions=10)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import psutil

# Monitor memory usage
def print_memory_usage():
    process = psutil.Process()
    print(f"Memory usage: {process.memory_info().rss / 1024 / 1024:.2f} MB")

# Before processing
print_memory_usage()

# Set up progress bar
with ProgressBar():
    # Process your DataFrame
    result = sampled_users_book.compute()  # or whatever operation you're doing
    
    # Save to parquet with optimizations
    result.to_parquet(
        'data/sampled_users_book.parquet',
        compression='snappy',
        index=False,
        engine='pyarrow'
    )

# After processing
print_memory_usage()